In [2]:
from collections import namedtuple
from math import sqrt, floor
from typing import List
from numpy import linalg as ln
import numpy as np

Vector = namedtuple("Vector", "v")

def Vector__getitem__(self: Vector, i: int):
    return self.v[i]
def Vector__setitem__(self: Vector, i: int, a: int):
    self.v[i] = a
def Vector__len__(self: Vector):
    return len(self.v)
def Vector__size__(self: Vector):
    return sqrt(self*self)
def Vector__add__(a: Vector, b: Vector):
    return Vector([a[i]+b[i] for i in range(len(a))])
def Vector__sub__(a: Vector, b: Vector):
    return Vector([a[i]-b[i] for i in range(len(a))])
def Vector__mul__(a: Vector, b: Vector):
    return sum([a[i]*b[i] for i in range(len(a))])
def Vector__rmul__(v: Vector, n: int):
    return Vector([n*v[i] for i in range(len(v))])
def Vector__truediv__(v: Vector, n: int):
    return Vector([v[i]/n for i in range(len(v))])

Basis = namedtuple("Basis", "V")
def Basis__getitem__(self: Basis, i: int):
    return self.V[i]
def Basis__setitem__(self: Basis, i: int, a: Vector):
    self.V[i] = a
def Basis__len__(self: Basis):
    return len(self.V)
def Basis_gram_schmidt_normalized(V: Basis):
    U = Basis([Vector([0]*len(V[0]))]*len(V))
    for i in range(0,len(V)):
        U[i] = V[i]
        for j in range(i):
            U[i] -= (V[i]*U[j])*U[j]
        U[i] /= sqrt(U[i]*U[i])
    return U

def Basis_gram_schmidt(V: Basis):
    U = Basis([Vector([0]*len(V[0]))]*len(V))
    for i in range(0,len(V)):
        U[i] = V[i]
        for j in range(i):
            U[i] -= ((V[i]*U[j])/(U[j]*U[j]))*U[j]
    return U

def Basis_lattice_point(V: Basis, coeff: List[int]):
    assert len(V)==len(coeff), "lengths of coefficients and basis must be the same"
    return [coeff[i]*V[i] for i in range(len(coeff))]
def Basis_determinant(V: Basis):
    return round(abs(ln.det(np.array([[j for j in i.v] for i in V.V]))))

Vector.__getitem__ = Vector__getitem__
Vector.__setitem__ = Vector__setitem__
Vector.__len__ = Vector__len__
Vector.__add__ = Vector__add__
Vector.__sub__ = Vector__sub__
Vector.__mul__ = Vector__mul__
Vector.__rmul__ = Vector__rmul__
Vector.__truediv__ = Vector__truediv__
Vector.size = Vector__size__

Basis.gs = Basis_gram_schmidt
Basis.gs_norm = Basis_gram_schmidt_normalized
Basis.det = Basis_determinant
Basis.lattice_point = Basis_lattice_point
Basis.__getitem__ = Basis__getitem__
Basis.__setitem__ = Basis__setitem__
Basis.__len__ = Basis__len__

def gaussian_lat_red(v1: Vector, v2: Vector):
    while True:
        if v2.size()<v1.size(): v2, v1 = v1, v2
        m = floor((v1*v2)/(v1*v1))
        if m<0: assert False, "failed to reduce lattice"
        if m==0: return v1, v2
        v2 -= m*v1

v1 = Vector([846835985,9834798552])
v2 = Vector([87502093,123094980])
print(Basis([v1, v2]).det())
_v1, _v2 = gaussian_lat_red(v1, v2)
print(Basis([_v1, _v2]).det())

756324198896513920
756324198896513920


In [6]:
from Crypto.Util.number import inverse, long_to_bytes
q, h = (7638232120454925879231554234011842347641017888219021175304217358715878636183252433454896490677496516149889316745664606749499241420160898019203925115292257, 2163268902194560093843693572170199707501787797497998463462129592239973581462651622978282637513865274199374452805292639586264791317439029535926401109074800)
e = 5605696495253720664142881956908624307570671858477482119657436163663663844731169035682344974286379049123733356009125671924280312532755241162267269123486523
f, g = gaussian_lat_red(Vector([1, h]), Vector([0, q]))[0].v
a = (f*e) % q
m = (a*inverse(f, g)) % g
long_to_bytes(m)

b'crypto{Gauss_lattice_attack!}'

In [ ]:
# %q
# h = f^-1*g
# e = rh + m = rf^-1*g + m
# a = fe = rg+fm

# %g
# m = f^-1a